In [2]:
import h5py
import numpy as np
import os
from PIL import Image
import io
import pandas as pd
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from numba import jit
from matplotlib import pyplot as plt


In [19]:
for x in range(0,100):
    save_path = '../hdf/images_data_train_0.h5'
    f = h5py.File(save_path, 'r')
train = pd.read_csv('../train.csv')
X = train['image']
y = train['class']   


In [21]:
x = range(X.shape[0])
l = np.array_split(np.array(x), 100)
z = 0

df = pd.DataFrame()
images = []
labels = []
for x in tqdm(l[0]):
    images.append(f['image '+str(x)][:]) 
    labels.append(f['label '+str(x)][()].decode())



100%|██████████| 9357/9357 [00:52<00:00, 178.99it/s]


In [22]:
df['labels'] = labels


In [23]:
y = df['labels']
X = np.array(images)

In [24]:
X.shape

(9357, 256, 256, 3)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [26]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [27]:
base_model = VGG16(weights='imagenet', include_top=False)

In [28]:
X_train = base_model.predict(X_train)
X_train.shape

(7485, 8, 8, 512)

In [29]:
X_test = base_model.predict(X_test)
X_test.shape

(1872, 8, 8, 512)

In [30]:
X_train = X_train.reshape(7485, 8*8*512)
X_test = X_test.reshape(1872, 8*8*512)

In [31]:
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [32]:
X_train.shape


(7485, 32768)

In [33]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

In [34]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('../models/weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [35]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [37]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/5
59/59 [==============================] - 8s 132ms/step - loss: 3.4547e-05 - accuracy: 1.0000 - val_loss: 3.9668e-10 - val_accuracy: 1.0000
Epoch 2/5
59/59 [==============================] - 8s 134ms/step - loss: 1.1962e-05 - accuracy: 1.0000 - val_loss: 2.7418e-10 - val_accuracy: 1.0000
Epoch 3/5
59/59 [==============================] - 8s 132ms/step - loss: 2.8561e-05 - accuracy: 1.0000 - val_loss: 1.1034e-10 - val_accuracy: 1.0000
Epoch 4/5
59/59 [==============================] - 8s 141ms/step - loss: 8.1250e-06 - accuracy: 1.0000 - val_loss: 4.3384e-11 - val_accuracy: 1.0000
Epoch 5/5
59/59 [==============================] - 8s 135ms/step - loss: 2.5575e-06 - accuracy: 1.0000 - val_loss: 3.0560e-11 - val_accuracy: 1.0000


In [41]:
model2 = Sequential()
model2.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(Dropout(0.5))
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(2, activation='sigmoid'))

In [42]:
model2.load_weights('../models/weight.hdf5')

In [43]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              33555456  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              